In [160]:
import os
import pandas as pd
import numpy as np
from tbparse import SummaryReader

# log_dir = os.path.expanduser("~/lis-cluster/emergent_communication/emergent_communication/lightning_logs")
# log_dir = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs_cluster")
log_dir = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs")

reader = SummaryReader(log_dir, extra_columns={'dir_name'}, pivot=True)
df = reader.scalars
hp = reader.hparams

# Save backups:
df_original = df.copy()
hp_original = hp.copy()

In [161]:
df = df_original.copy()
hp = hp_original.copy()

In [162]:
def fix_duplicate_value(val, allow_offset=None):
    if isinstance(val, list):
        for el in val:
            if allow_offset is None:
                assert (el == val[0]) or (el == "None") or (val[0] == "None")
            else:
                assert (np.abs(el - val[0]) < allow_offset) or (el == "None") or (val[0] == "None")
        return val[0]
    else:
        return val

def flatten_lists(val):
    if isinstance(val, list):
        return val[-1]
    else:
        return val

hp = hp.applymap(fix_duplicate_value)
# For val_acc, results may vary because of noise
df["val_acc"] = df["val_acc"].apply(flatten_lists)
df["val_acc_no_noise"] = df["val_acc_no_noise"].apply(fix_duplicate_value)
df

,step,best_val_acc_no_noise,bosdis,bosdis_at_best_val_acc,entropy_loss,entropy_loss/dataloader_idx_0,entropy_loss/dataloader_idx_1,entropy_loss/dataloader_idx_2,epoch,hp_metric,...,test_acc,test_acc_no_noise,topsim,topsim_at_best_val_acc,train_acc,train_acc_no_noise,train_acc_no_noise_at_best_val_acc,val_acc,val_acc_no_noise,dir_name
0,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,"[-1.0, -1.0, -1.0]",...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000,version_915
1,49,NaN,NaN,NaN,1.380435,NaN,NaN,NaN,49.0,NaN,...,NaN,NaN,NaN,NaN,0.004831,NaN,NaN,NaN,NaN,version_915
2,99,0.0,NaN,NaN,1.380056,1.378673,1.37844,1.378409,"[99.0, 99.0]",NaN,...,0.0,0.0,NaN,NaN,0.000000,0.009662,NaN,0.000000,0.000000,version_915
3,149,NaN,NaN,NaN,1.349310,NaN,NaN,NaN,149.0,NaN,...,NaN,NaN,NaN,NaN,0.009662,NaN,NaN,NaN,NaN,version_915
4,199,0.0,NaN,NaN,1.320210,1.327956,1.337823,1.327906,"[199.0, 199.0]",NaN,...,0.0,0.0,NaN,NaN,0.009662,0.009662,NaN,0.000000,0.000000,version_915
5,249,NaN,NaN,NaN,1.286145,NaN,NaN,NaN,249.0,NaN,...,NaN,NaN,NaN,NaN,0.014493,NaN,NaN,NaN,NaN,version_915
6,299,0.0,NaN,NaN,1.278823,1.308884,1.316238,1.308742,"[299.0, 299.0]",NaN,...,0.0,0.0,NaN,NaN,0.019324,0.028986,NaN,0.000000,0.000000,version_915
7,349,NaN,NaN,NaN,1.257812,NaN,NaN,NaN,349.0,NaN,...,NaN,NaN,NaN,NaN,0.038647,NaN,NaN,NaN,NaN,version_915
8,399,0.043478,NaN,NaN,1.252917,1.280639,1.287605,1.27751,"[399.0, 399.0]",NaN,...,0.0,0.0,NaN,NaN,0.033816,0.062802,0.062802,0.043478,0.043478,version_915
9,449,NaN,NaN,NaN,1.237628,NaN,NaN,NaN,449.0,NaN,...,NaN,NaN,NaN,NaN,0.057971,NaN,NaN,NaN,NaN,version_915


In [159]:
# Keep only best step values for each run
df.dropna(subset=["val_acc", "topsim"], inplace=True)
df["val_acc_no_noise"] = df["val_acc_no_noise"].astype(float)
df["val_acc"] = df["val_acc"].astype(float)

indices_best_steps = set(pd.concat((df.groupby(["dir_name"])["val_acc_no_noise"].idxmax(), df.groupby(["dir_name"])["val_acc"].idxmax())))
print(indices_best_steps)
df = df.loc[list(indices_best_steps)]
df

{23}


,step,best_val_acc_no_noise,bosdis,bosdis_at_best_val_acc,entropy_loss,entropy_loss/dataloader_idx_0,entropy_loss/dataloader_idx_1,entropy_loss/dataloader_idx_2,epoch,hp_metric,...,test_acc,test_acc_no_noise,topsim,topsim_at_best_val_acc,train_acc,train_acc_no_noise,train_acc_no_noise_at_best_val_acc,val_acc,val_acc_no_noise,dir_name
23,1116,"[0.08695652335882187, 0.08695652335882187]","[0.07015444338321686, 0.06852047890424728]","[0.07015444338321686, 0.06852047890424728]",NaN,"[1.0937448740005493, 1.1130340099334717]","[1.132897138595581, 1.1362501382827759]","[1.1138365268707275, 1.1186163425445557]","[1116.0, 1116.0]",NaN,...,"[0.03846153989434242, 0.07692307978868484]","[0.03846153989434242, 0.03846153989434242]","[0.4211689829826355, 0.44422784447669983]","[0.4211689829826355, 0.44422784447669983]",NaN,"[0.4492753744125366, 0.3719806671142578]","[0.4492753744125366, 0.3719806671142578]",0.043478,0.086957,version_915


In [151]:
# Now remove all duplicate values from the smaller dataframe
def flatten_lists(val):
    if isinstance(val, list):
        if val[-1] is None or val[-1] == "NaN":
            return val[0]
        else:
            return val[-1]
    else:
        return val
df = df.applymap(flatten_lists)
df

,step,best_val_acc_no_noise,bosdis,bosdis_at_best_val_acc,entropy_loss,entropy_loss/dataloader_idx_0,entropy_loss/dataloader_idx_1,entropy_loss/dataloader_idx_2,epoch,hp_metric,...,test_acc,test_acc_no_noise,topsim,topsim_at_best_val_acc,train_acc,train_acc_no_noise,train_acc_no_noise_at_best_val_acc,val_acc,val_acc_no_noise,dir_name
23,1116,0.086957,0.06852,0.06852,NaN,1.113034,1.13625,1.118616,1116.0,NaN,...,0.076923,0.038462,0.444228,0.444228,NaN,0.371981,0.371981,0.043478,0.086957,version_915


In [152]:
df.set_index("dir_name", inplace=True, drop=False)
hp.set_index("dir_name", inplace=True, verify_integrity=True)

df = df.join(hp, how="left")

In [153]:
df.to_pickle("df.pickle")

In [154]:
df


,step,best_val_acc_no_noise,bosdis,bosdis_at_best_val_acc,entropy_loss,entropy_loss/dataloader_idx_0,entropy_loss/dataloader_idx_1,entropy_loss/dataloader_idx_2,epoch,hp_metric,...,sync_batchnorm,terminate_on_nan,test_set_size,tpu_cores,track_grad_norm,val_check_interval,vocab_size,vocab_size_feedback,weights_save_path,weights_summary
dir_name,,,,,,,,,,,,,,,,,,,,,
version_915,1116,0.086957,0.06852,0.06852,NaN,1.113034,1.13625,1.118616,1116.0,NaN,...,0.0,None,0.1,None,-1.0,None,100.0,2.0,None,top
